## 6. Fixtures for setup / teardown

In pytest, **fixtures** give reusable setup + cleanup logic while keeping test code concise. Declare with `@pytest.fixture` and inject by parameter name into any test function.

```python
import pytest, tempfile, shutil, os

@pytest.fixture
def temp_dir():
    path = tempfile.mkdtemp()
    yield path              # tests run here
    shutil.rmtree(path)     # teardown

def test_write(temp_dir):
    file = os.path.join(temp_dir, 'hello.txt')
    open(file, 'w').write('hi')
    assert open(file).read() == 'hi'
```

### Quick check

1. Yield inside fixture marks:
  a. teardown before  b. split setup/teardown

2. True / False Fixture scope can be session‑wide.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – set `scope='session'`.

</details>

## 7. Parametrised tests

`@pytest.mark.parametrize` generates multiple cases from data table, avoiding loops inside tests and producing independent pass/fail results.

```python
import pytest
@pytest.mark.parametrize('a,b,expected',[ (2,3,5), (10,5,15) ])
def test_add(a,b,expected):
    assert a+b == expected
```

### Quick check

1. Parametrisation shows each case as:
  a. one combined test  b. separate tests

2. True / False You can parametrize over fixtures.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – use `indirect=True`.

</details>

## 8. Mocking basics

Replace slow or external collaborators with **mocks**. With `unittest.mock`: `patch('module.func')` swaps during test. Use `Mock()` to assert call count and arguments.

```python
from unittest.mock import patch
import requests

def fetch_json(url):
    return requests.get(url).json()

def test_fetch():
    with patch('requests.get') as mock_get:
        mock_get.return_value.json.return_value = {'ok':True}
        assert fetch_json('x')['ok']
        mock_get.assert_called_once()
```

### Quick check

1. `patch()` should reference:
  a. import location in SUT  b. original library path

2. True / False `Mock()` records attribute access automatically.

<details><summary>Answer key</summary>

1. **a** – patch where used.
2. **True** – dynamic mocks.

</details>

## 9. Golden files & snapshot testing

For large text/binary outputs you can store **golden files**. Test compares current output to file checked into repo. Tools like `pytest-snapshot` manage approve/update workflow.

```python
def render():
    return '<html>hi</html>'

def test_snapshot(snapshot):
    snapshot.assert_match(render(), 'page.html')
```

### Quick check

1. Golden tests fail when output:
  a. equals golden  b. differs

2. True / False Updating golden should be code‑reviewed.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 10. Isolating external resources

Keep tests deterministic by faking I/O: in‑memory SQLite, moto for AWS S3, httpx `MockTransport` for HTTP. Ensures CI runs offline & free.

```python
from moto import s3
import boto3, pytest

@pytest.fixture(autouse=True)
def mock_aws():
    with s3.mock_s3():
        yield

def test_upload():
    s = boto3.client('s3', region_name='us-east-1')
    s.create_bucket(Bucket='test')
    s.put_object(Bucket='test', Key='file', Body=b'data')
    assert s.list_objects(Bucket='test')['KeyCount'] == 1
```

### Quick check

1. Using real AWS in CI may cause:
  a. cost & flake  b. faster tests

2. True / False moto intercepts boto3 calls without network.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>